In [ ]:
import pandas as pd

# 파일 읽기
file_path = '/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/kmhas_data/kmhas_raw.txt'
df = pd.read_csv(file_path, sep='\t')

# 새로운 라벨 열을 생성
labels = ["출신차별", "외모차별", "정치성향차별", "혐오욕설", "연령차별", "성차별", "인종차별", "종교차별", "해당사항 없음"]
label_map = {0: "출신차별", 1: "외모차별", 2: "정치성향차별", 3: "혐오욕설", 4: "연령차별", 5: "성차별", 6: "인종차별", 7: "종교차별", 8: "해당사항 없음"}

# 기존 라벨들을 0과 1로 변환
for label in labels:
    df[label] = 0

# 기존 라벨을 ,로 분리하여 새로운 라벨 값 설정
for index, row in df.iterrows():
    if not pd.isna(row['label']):
        for label in map(int, row['label'].split(',')):
            label_name = label_map[label]
            df.at[index, label_name] = 1

# 불필요한 기존 라벨 열 제거
df.drop(columns=['label'], inplace=True)

# 파일 저장
output_path = '/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/kmhas_data/kmhas_dataset.csv'
df.to_csv(output_path, sep='\t', index=False)


In [17]:
import pandas as pd

df = pd.read_csv('/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/kmhas_data/kmhas_dataset.txt',delimiter = '\t',on_bad_lines='skip')

In [18]:
df.head(10)

,document,출신차별,외모차별,정치성향차별,혐오욕설,연령차별,성차별,인종차별,종교차별,해당사항 없음
0,자한당틀딱들.. 악플질 고만해라.,0,0,1,0,1,0,0,0,0
1,정치적으로 편향된 평론한은 분은 별로...,0,0,0,0,0,0,0,0,1
2,적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...,0,0,1,0,0,0,0,0,0
3,안서는 아재들 풀발기 ㅋㄲㅋ,0,0,0,0,1,0,0,0,0
4,우와 ㅋ 능력자,0,0,0,0,0,0,0,0,1
5,맛녀석 콩트보다 약했음맛녀석 애청자로써 70%실력발휘,0,0,0,0,0,0,0,0,1
6,주영훈 솔직히 호감임 잉꼬부부로 소문났잖아,0,0,0,0,0,0,0,0,1
7,이게주간아이돌이랑머가달라...,0,0,0,0,0,0,0,0,1
8,아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사...,0,0,0,1,0,0,0,0,0
9,문재인 하는게 뭐 별거있냐?ㅂㅅㅅㅋ가 하는짓인데 어련하겠어.ㅋㅋㅋ,0,0,1,1,0,0,0,0,0


In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = '/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/kmhas_data/kmhas_dataset.txt'
data = pd.read_csv(file_path, delimiter='\t')

# Display the first few rows of the dataset
print(data.head())

# Define the labels
labels = ['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '해당사항 없음']

# Text cleaning function
def clean_text(text):
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"[^a-zA-Z가-힣0-9\s]", "", text)
    return text.strip()

# Clean the 'document' column
data['document'] = data['document'].apply(clean_text)

# Convert labels to a list of labels
def extract_labels(row):
    return [labels[i] for i in range(len(labels)) if row[i+1] == 1]

data['labels'] = data.apply(extract_labels, axis=1)

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['document'])
sequences = tokenizer.texts_to_sequences(data['document'])

# Pad the sequences
maxlen = 100  # You can adjust the maxlen according to your needs
X = pad_sequences(sequences, maxlen=maxlen)

# Binarize the labels
mlb = MultiLabelBinarizer(classes=labels)
y = mlb.fit_transform(data['labels'])

# Print the shape of the dataset
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')



                                    document  출신차별  외모차별  정치성향차별  혐오욕설  연령차별  \
0                         자한당틀딱들.. 악플질 고만해라.     0     0       1     0     1   
1                    정치적으로 편향된 평론한은 분은 별로...     0     0       0     0     0   
2  적당히좀 쳐먹지.그랬냐??? 안그래도 문재인 때문에 나라 엉망진창인데...     0     0       1     0     0   
3                            안서는 아재들 풀발기 ㅋㄲㅋ     0     0       0     0     1   
4                                   우와 ㅋ 능력자     0     0       0     0     0   

   성차별  인종차별  종교차별  해당사항 없음  
0    0     0     0        0  
1    0     0     0        1  
2    0     0     0        0  
3    0     0     0        0  
4    0     0     0        1  


/var/folders/sd/pq30895j7xl996rt4n_pcm0r0000gn/T/ipykernel_79044/329801876.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return [labels[i] for i in range(len(labels)) if row[i+1] == 1]


Shape of X: (109692, 100)
Shape of y: (109692, 9)


In [35]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 283379
등장 빈도가 2번 이하인 희귀 단어의 수: 248418
단어 집합에서 희귀 단어의 비율: 87.66281199383158
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 36.01167770884504


In [36]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 34963
